<a href="https://colab.research.google.com/github/Lucasdz-7/Projeto-Integrador-2/blob/main/C%C3%B3digoAPi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import os
import re
from google.colab import files
from google.colab import drive

In [44]:
drive.mount('/content/drive')
origem = '/content/drive/MyDrive/Data/Dados/'
DadosIbama = origem + 'dados_ibama.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Criando Dataframes

Dados_IBAMA = pd.read_csv(DadosIbama, low_memory=False, sep=';', encoding='utf-8')

In [ ]:
Dados_IBAMA.head(10)

In [46]:
# Filtrando para o período entre 2013 - 2023 e ordenando por ordem crescente

Dados_IBAMA_2013a2023 = Dados_IBAMA[(Dados_IBAMA["Ano"] >= 2013) & (Dados_IBAMA["Ano"] <= 2023)]
Dados_IBAMA_2013a2023 = Dados_IBAMA_2013a2023.sort_values(by='Ano', ascending=True)

In [ ]:
Dados_IBAMA_2013a2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2941968 entries, 200354 to 281960
Data columns (total 16 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   CNPJ                     object
 1   Razão Social             object
 2   Estado                   object
 3   Município                object
 4   Ano                      int64 
 5   Produto                  object
 6   Quantidade Transportada  object
 7   Unidade de Medida        object
 8   Tipo de Transporte       object
 9   Tipo de Armazenamento    object
 10  Plano de Emergência      object
 11  UF - origem              object
 12  Município - origem       object
 13  UF - destino             object
 14  Município - destino      object
 15  Situação Cadastral       object
dtypes: int64(1), object(15)
memory usage: 381.6+ MB


In [ ]:
Dados_IBAMA_2013a2023.head(5)

In [47]:
# Removendo todos outros modais, preservando apenas o rodoviário.

Dados_IBAMA_teste = Dados_IBAMA_2013a2023.query("`Tipo de Transporte` == 'Rodoviário'")

In [48]:
# Removendo zeros e valores nulos.

Dados_IBAMA_teste = Dados_IBAMA_teste[(Dados_IBAMA_teste != 0).all(axis=1)].dropna()
(Dados_IBAMA_teste == 0).sum()

,0
CNPJ,0
Razão Social,0
Estado,0
Município,0
Ano,0
Produto,0
Quantidade Transportada,0
Unidade de Medida,0
Tipo de Transporte,0
Tipo de Armazenamento,0


In [49]:
# Apenas unidades de medidas de kg e litros

Dados_IBAMA_teste = Dados_IBAMA_teste.query(
    "`Unidade de Medida` in ['kilogramas', 'Litro', 'Metro Cúbico',  'Tonelada']"
)

In [ ]:
# Conferindo quais unidades de medida restaram

print(Dados_IBAMA_teste['Unidade de Medida'].unique())

['Tonelada' 'kilogramas' 'Litro' 'Metro Cúbico']


In [53]:
# criando colunas novas
Dados_IBAMA_teste['qntd_Litros'] = 0.0
Dados_IBAMA_teste['qntd_Kg'] = 0.0

# Os dados estavam no formato de str, essa etapa do código corrige e normaliza os valores
Dados_IBAMA_teste['Quantidade Transportada'] = Dados_IBAMA_teste['Quantidade Transportada'].apply(
    lambda x: float(str(x).replace('.', '').replace(',', '.')) if x not in [None, ''] else 0.0
)

def normalizar(row):
    valor = row['Quantidade Transportada']
    unidade = str(row['Unidade de Medida']).strip()

    # --- CONVERSÃO PARA LITRO ---
    if unidade in ['Metro Cúbico']:
        row['Quantidade Transportada'] = valor * 1000
        row['Unidade de Medida'] = 'Litro'

    # --- CONVERSÃO PARA KILOGRAMA ---
    elif unidade in ['Tonelada']:
        # Mantém o valor original (não multiplica, pois analisando os dados, percebemos que os valores de tonelada na verdade estão representados em quilogramas, não sendo necessário aplicar conversão.)
        row['Unidade de Medida'] = 'kilogramas'

    return row

# Aplicando a normalização
Dados_IBAMA_teste = Dados_IBAMA_teste.apply(normalizar, axis=1)

# Preenche as colunas que foram criadas anteriormente
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'Litro', 'qntd_Litros'] = Dados_IBAMA_teste['Quantidade Transportada']
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'kilogramas', 'qntd_Kg'] = Dados_IBAMA_teste['Quantidade Transportada']

# Apagando as colunas de unidade de medida e quantidade transportada
Dados_IBAMA_teste = Dados_IBAMA_teste.drop(columns=['Quantidade Transportada', 'Unidade de Medida'])

print(Dados_IBAMA_teste[['qntd_Litros', 'qntd_Kg']].head(10))


        qntd_Litros  qntd_Kg
200354          0.0  30951.0
200353          0.0  25978.0
200352          0.0  13190.0
200288          0.0  34830.0
200269          0.0  14139.0
200225          0.0  13840.0
200121          0.0  31222.0
198898          0.0  25949.0
198535          0.0  13960.0
198498          0.0   7170.0


In [54]:
Dados_IBAMA_teste.tail(10)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral,qntd_Litros,qntd_Kg
2743289,46.720.027/0001-50,3R COMERCIO E LOGISTICA LTDA,SAO PAULO,RIBEIRAO PRETO,2023,Serviço de produção de álcool ou serviços rela...,Rodoviário,Tanques,Sim,SAO PAULO,GUARULHOS,SAO PAULO,BATATAIS,Ativa,216000.0,0.0
3788396,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Ar líqüido ou ar comprimido,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,IVAIPORA,Ativa,2000.0,0.0
3787825,00.712.503/0001-90,OXBRAS GASES MEDICINAIS E INDUSTRIAIS DO NORDE...,BAHIA,ALAGOINHAS,2023,Oxigênio,Rodoviário,Tanques,Sim,BAHIA,ALAGOINHAS,BAHIA,ARACAS,Ativa,13500000.0,0.0
3787826,00.712.503/0001-90,OXBRAS GASES MEDICINAIS E INDUSTRIAIS DO NORDE...,BAHIA,ALAGOINHAS,2023,Argônio (gases raros),Rodoviário,Tanques,Sim,BAHIA,ALAGOINHAS,BAHIA,RIO REAL,Ativa,37000.0,0.0
3787904,01.112.425/0001-55,MAGMIN LTDA,MINAS GERAIS,ITAUNA,2023,"Explosivos preparados, exceto pólvoras propuls...",Rodoviário,Outros,Sim,MINAS GERAIS,ITAUNA,MINAS GERAIS,DESTERRO DE ENTRE RIOS,Ativa,0.0,2575.0
3788372,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MANDAGUARI,Ativa,415000.0,0.0
3787973,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,"Produtos inorgânicos, n.e, exceto os compostos...",Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,MARINGA,Ativa,603000.0,0.0
3788169,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Acetileno,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,PATO BRANCO,Ativa,0.0,33.0
3788355,05.084.081/0001-79,F C DE PAULA JOSE & CIA LTDA,PARANA,PARANAVAI,2023,Oxigênio,Rodoviário,Outros,Sim,PARANA,PARANAVAI,PARANA,TUNEIRAS DO OESTE,Ativa,190000.0,0.0
281960,09.446.066/0001-84,OXICAM GASES LTDA ME,SAO PAULO,PAULINIA,2023,Oxigênio,Rodoviário,Tanques,Não,ESPIRITO SANTO,VITORIA,SAO PAULO,CAMPINAS,Ativa,0.0,30000.0


In [58]:
import re

palavras_perigosas = [
    # Combustíveis e derivados
    'gasolina', 'diesel', 'etanol', 'álcool', 'biodiesel', 'combustível', 'querosene',
    'gnv', 'gás liquefeito', 'gás natural', 'butano', 'propano', 'glp', 'naphta',
    'nafta', 'óleo mineral', 'lubrificante',

    # Substâncias químicas
    'ácido', 'base', 'alcalino', 'hidróxido', 'nitrato', 'peróxido', 'hipoclorito',
    'cloro', 'amônia', 'soda cáustica', 'cloreto', 'sulfato', 'fosfato',
    'ácetico', 'acético', 'cianeto', 'anidro', 'solvente', 'tolueno', 'xileno',
    'benzeno', 'acetona', 'formaldeído', 'fenol', 'butil', 'metanol', 'isopropanol',
    'hexano', 'heptano', 'éter',

    # Termos de risco
    'inflamável', 'tóxico', 'corrosivo', 'explosivo', 'oxidante', 'radioativo',
    'perigoso', 'reativo', 'irritante', 'comburente', 'poluente',

    # Resíduos e efluentes
    'resíduo', 'lama', 'lodo', 'efluente', 'emulsão', 'borra', 'catalisador',
    'defensivo', 'fertilizante químico', 'pesticida', 'herbicida', 'inseticida',

    # Gases e criogênicos
    'oxigênio', 'nitrogênio', 'hélio', 'argônio', 'co2', 'co', 'freon',
    'refrigerante industrial', 'criogenico', 'criogênico', 'gás comprimido',

    # Metais pesados
    'mercúrio', 'chumbo', 'cádmio', 'arsênio', 'cobre', 'níquel',
    'zinco', 'manganês', 'cromato', 'dicromato',

    # Produtos industriais
    'betume', 'asfalto', 'tinta', 'verniz', 'cola industrial', 'adesivo químico',
    'detergente industrial', 'espessante químico', 'aditivo químico',
    'borracha sintética', 'plástico fundido'
]

# Cria regex com fronteiras de palavra (\b) e escapa os termos especiais
pattern = r'\b(?:' + '|'.join(re.escape(p) for p in palavras_perigosas) + r')\b'

#  Aplica a máscara do filtro
mask = Dados_IBAMA_teste['Produto'].str.lower().str.contains(pattern, flags=re.IGNORECASE, na=False)

# Filtrando o dataframe
Dados_IBAMA_filtrado = Dados_IBAMA_teste[mask].copy()


In [61]:
contagem_valores = Dados_IBAMA_filtrado['Produto'].value_counts()
print(contagem_valores)

Produto
Óleo diesel                                                                                                                     225968
Álcool etílico não desnaturado, com teor alcoólico em volume maior ou igual a 80%, anidro ou hidratado para fins carburantes    202832
Gasolina automotiva ou para outros usos, exceto para aviação                                                                    106910
Gasolina automotiva formulada                                                                                                    87796
Gás liqüefeito de petróleo (GLP)                                                                                                 70488
                                                                                                                                 ...  
Cordas, cabos, tranças e artefatos semelhantes de cobre, não isolados                                                                1
Ligas mãe de cobre                             

In [62]:
Dados_IBAMA_filtrado['Produto'].value_counts().head(50)

,count
Produto,
Cátodos de níquel não ligado forma bruta,3
Misturas de uréia com nitrato de amônio em soluções aquosas ou amoniac,3
"Serviço de revestimento, à base de borracha, em cilindros e outras peças para máquinas e veículos",3
Pigmentos orgânicos e preparações a base desses pigmentos (masterbache,3
Barras de liga de níquel,3
Ferrossilício-manganês,3
"Preparações alimentares à base de farinhas, sêmolas e etc.",3
"Compostos organo-inorgânicos, n.e, exceto compostos organoestánicos(co",3
Medicamentos n.e. à base de alcalóides ou seus derivados - (uso veterinário),2


In [66]:
# Apagando colunas desnecessárias (até o momento) e reiniciando o índice

DadosIBAMA_final = Dados_IBAMA_filtrado.drop(columns=['Tipo de Transporte', 'Tipo de Armazenamento'])
DadosIBAMA_final = DadosIBAMA_final.reset_index(drop=True)

In [67]:
DadosIBAMA_final.head(10)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral,qntd_Litros,qntd_Kg
0,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa,0.0,30951.0
1,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa,0.0,25978.0
2,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,COTIA,MINAS GERAIS,BELO HORIZONTE,Ativa,0.0,13190.0
3,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,COTIA,RIO DE JANEIRO,RESENDE,Ativa,0.0,34830.0
4,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,COTIA,MINAS GERAIS,UBERLANDIA,Ativa,0.0,14139.0
5,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,COTIA,MINAS GERAIS,SAO GONCALO DO SAPUCAI,Ativa,0.0,13840.0
6,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,GUARUJA,SAO PAULO,COTIA,Ativa,0.0,31222.0
7,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,GUARULHOS,MINAS GERAIS,UBERABA,Ativa,0.0,25949.0
8,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,SUZANO,SAO PAULO,LIMEIRA,Ativa,0.0,13960.0
9,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Sim,SAO PAULO,GUARULHOS,GOIAS,APARECIDA DE GOIANIA,Ativa,0.0,31010.0


In [68]:
# Salvando o arquivo principal
DadosIBAMA_final.to_csv("DadosIBAMA_final.csv", sep=";", index=False, encoding="utf-8")

print(os.listdir())
files.download("DadosIBAMA_final.csv")

['.config', 'DadosIBAMA_final.csv', 'drive', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>